## Historic Betting Info for Threshold Calculation 

In [8]:
import requests
import pandas as pd
import numpy as np 
def fetch_betting_data():
    #API Key found online for accessing BettingPros data 
    url = "https://api.bettingpros.com/v3/pbcs"
    params = {
        "sport": "NBA",
        "event_id": "21353:21354:21355:21356:21357",
        "market_id": "151:152:156:157:160:162",
        "book_id": "10:12:13:14:15:18:19:20:21:22:24:25:26:27:28:29:30:31",
        "location": "ALL"
    }

    headers = {
        "x-api-key": "CHi8Hy5CEE4khd46XNYL23dCFX96oUdw6qOt1Dnh"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Error: {response.status_code}")

def extract_betting_info(data):
    market_data = []
    
    for market in data.get('markets', []):
        for offer in market.get('offers', []):
            player_info = offer.get('participant', {}).get('player', {})
            market_data.append({
                ##Add the necessary columns
                'Team': player_info.get('team'),
                'Position': player_info.get('position'),
                'Projection': offer.get('projection'),
                'Recommendation': offer.get('recommendation'),
                'Line': offer.get('line'),
                'Actual': offer.get('scoring', {}).get('actual'),
                'Market Label': market.get('market_full_label')
            })

    return pd.DataFrame(market_data)

# Fetch data
data = fetch_betting_data()

# Extract and convert to DataFrame
df = extract_betting_info(data)


df["Deviation"] = np.abs(df["Projection"] - df["Actual"])  

def determine_correctness(row):
    if row['Recommendation'] == "over" and row['Actual'] > row['Line']:
        return "right"
    elif row['Recommendation'] == "under" and row['Actual'] < row['Line']:
        return "right"
    else:
        return "wrong"

# Apply the correctness function
df["Correctness"] = df.apply(determine_correctness, axis=1)



In [6]:
df.to_csv("NBA_BettingData.csv") 

In [7]:
df

,Team,Position,Projection,Recommendation,Line,Actual,Market Label,Deviation,Correctness
0,NYK,F,9,over,4.5,3,Rebounds,6,wrong
1,ATL,C,3,under,7.5,4,Rebounds,1,right
2,UTH,C,13,under,16.5,11,Rebounds,2,right
3,ATL,G,3,under,5.5,4,Rebounds,1,right
4,MIL,F,7,over,4.5,10,Rebounds,3,right
...,...,...,...,...,...,...,...,...,...
299,UTH,G,1,over,0.5,1,Steals,0,right
300,DAL,C,1,over,0.5,0,Steals,1,wrong
301,UTH,G,1,under,1.5,3,Steals,2,wrong
302,UTH,F,1,over,0.5,0,Steals,1,wrong
